In [227]:
from itertools import product
from random import randint, seed
import random
import numpy as np
from scipy import sparse
from functools import reduce
from random import random, choice, randint
from copy import copy
import matplotlib.pyplot as plt

In [228]:
def make_set_covering_problem(num_points, num_sets, density):
    """Returns a sparse array where rows are sets and columns are the covered items"""
    seed(num_points*2654435761+num_sets+density)
    sets = sparse.lil_array((num_sets, num_points), dtype=bool)
    for s, p in product(range(num_sets), range(num_points)):
        if random() < density:
            sets[s, p] = True
    for p in range(num_points):
        sets[randint(0, num_sets-1), p] = True
    return sets

Iterated Local Search

In [229]:
num_points = 5000
num_sets = num_points
density = .7
count_fitness = 0

mysets = make_set_covering_problem(num_points, num_sets, density).toarray()

def fitness(state):
    global count_fitness
    count_fitness = count_fitness + 1
    cost = sum(state)
    valid = np.sum(
        reduce(
            np.logical_or,
            [mysets[i] for i, t in enumerate(state) if t],
            np.array([False for _ in range(num_points)]),
        )
    )
    return valid, -cost



In [230]:
def tweak(state):
    new_state = copy(state)
    index = randint(0, num_points - 1)
    new_state[index] = not new_state[index]
    return new_state

def perturb(state):
    num_changes = np.sum(state) * .6
    new_state = copy(state)

    for _ in range(int(num_changes)):
        index = randint(0, num_points - 1)
        new_state[index] = not state[index]

    return new_state

    

In [231]:
current_state = [choice([False, False, False, False, False, False]) for _ in range(num_sets)]

current_home_base = copy(current_state)
current_home_base_fitness = fitness(current_home_base)
current_state_fitness = fitness(current_state)
timer=0

while current_home_base_fitness >= current_state_fitness and timer<3:
    timer+=1
    for _ in range(100):
        new_state = tweak(current_state)
        
        if (fitness(new_state) > current_state_fitness):
            current_state = copy(new_state) 
            current_state_fitness = fitness(current_state) # in order to not compute it each time...
            print(current_state_fitness)


    if current_home_base_fitness < current_state_fitness:
        timer=0
        current_home_base = copy(current_state)
        current_home_base_fitness = copy(current_state_fitness)

    current_state = perturb(current_home_base)
    current_state_fitness = fitness(current_state) # in order to not compute it each time...
    
print("")
print("Fitness calls: %s" % (count_fitness,))
print("Best fitness: %s" % (current_home_base_fitness,))
print("Best state: %s" % ([index for index, value in enumerate(current_home_base) if value]))



(1446, -1)
(2546, -2)
(3254, -3)
(3758, -4)
(4129, -5)
(4390, -6)
(4567, -7)
(4701, -8)
(4787, -9)
(4854, -10)
(4899, -11)
(4926, -12)
(4950, -13)
(4964, -14)
(4976, -15)
(4983, -16)
(4991, -17)
(4994, -18)
(4999, -19)
(5000, -20)
(5000, -31)
(5000, -31)

Fitness calls: 428
Best fitness: (5000, -20)
Best state: [519, 670, 688, 705, 1159, 1645, 2219, 2261, 2621, 2712, 2722, 3127, 3452, 3664, 3898, 4372, 4425, 4601, 4651, 4921]
